# beomi/KcELECTRA-base

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1835692544820087483
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2254123828
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9169485238189335735
physical_device_desc: "device: 0, name: GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [2]:
%run KcELECTRA-base.version.py
!pip freeze

라이브러리 설치가 완료되었습니다.
absl-py==2.0.0
accelerate==0.20.1
annotated-types==0.6.0
asttokens==2.4.1
astunparse==1.6.3
backcall==0.2.0
blis==0.7.11
cachetools==5.3.2
catalogue==2.0.10
certifi==2023.7.22
chardet==5.2.0
charset-normalizer==3.3.1
click==8.1.7
cloudpathlib==0.16.0
colorama==0.4.6
comm==0.1.4
confection==0.1.3
contourpy==1.1.1
cycler==0.12.1
cymem==2.0.8
dataclasses==0.6
debugpy==1.8.0
decorator==5.1.1
emoji==1.2.0
en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.0/en_core_web_sm-3.7.0-py3-none-any.whl#sha256=6215d71a3212690e9aec49408a27e3fe6ad7cd6c715476e93d70dc784041e93e
et-xmlfile==1.1.0
exceptiongroup==1.1.3
executing==2.0.0
filelock==3.12.4
flatbuffers==1.12
fonttools==4.43.1
fsspec==2023.10.0
gast==0.4.0
google-auth==2.23.3
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.59.0
h5py==3.1.0
huggingface-hub==0.17.3
idna==3.4
importlib-metadata==6.8.0
importlib-resources==6.1.0
iniconfig==2.0.0
install==1.3.5
ipykernel==6.26

In [3]:
import re
import torch
import pandas as pd

# deivce 선택
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("devcie:", device)

devcie: cpu


In [4]:
# 엑셀 파일에서 데이터프레임 읽기
df = pd.read_excel(r"C:\Users\GJAISCHOOL\Desktop\X_filter\Algorithm\dataset\asdf.xlsx")

# 'Sentence' 칼럼의 값을 문자열로 변환하여 리스트로 저장
str_data = df['Sentence'].astype(str).tolist()

# 'Sentence' 칼럼의 값을 하나의 문자열로 결합
all_sentences = ' '.join(str_data)

print(df.shape)
df.head()

(96, 2)


,Sentence,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1


In [5]:
null_idx = df[df.label.isnull()].index                             # 해당 index에 null 값 확인
df.loc[null_idx, "Sentence"]                                       # null 값이 존재한 인덱스의 content 값 불러오기

# lable은 content의 가장 끝 문장열로 설정
df.loc[null_idx, "label"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정
df.loc[null_idx, "Sentence"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-2])

In [7]:
train_data = df.sample(frac=0.8, random_state=42)                 # train(80%), test(20%) 셋 구분 
test_data = df.drop(train_data.index)                             # 랜덤으로 샘플링(랜덤으로 숫자 배치)

# 데이터셋 갯수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=["Sentence"], inplace=True)
test_data.drop_duplicates(subset=["Sentence"], inplace=True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))

In [8]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, TFElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# KcELECTRA 모델 및 토크나이저 로드
model_name = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
train_data

,Sentence,label,result,result_str
40,전에 제주도 여행 갔는데 폭포 앞에 사진 잘나오는 포토존에서 사진 찍는거 기다렸는데...,1,"[전, 제주도, 여행, 가다, 폭포, 앞, 사진, 잘나다, 포토, 존, 사진, 찍다...",전 제주도 여행 가다 폭포 앞 사진 잘나다 포토 존 사진 찍다 기다리다 앞 짱 게이...
67,진짜 몇시간씩 할 정도면 히로뽕맞고 하는거일수도있다경교대있을때 뽕쟁이 재소자 의무실...,1,"[진짜, 시간, 하다, 정도, 히로뽕, 맞다, 있다, 경교대, 때, 뽕, 재소자, ...",진짜 시간 하다 정도 히로뽕 맞다 있다 경교대 때 뽕 재소자 의무실 가다 계호 때 ...
15,대가리에 필터없는 연봉 30억 강사vs대가리가 없는 용접공,1,"[대가리, 필터, 없다, 연봉, 강사, 대가, 없다, 용접공]",대가리 필터 없다 연봉 강사 대가 없다 용접공
68,"휴게소, 마트 등의 여성전용주차장 보고 분노를 해야지",0,"[휴게소, 마트, 여성, 전용, 주차장, 보다, 분노, 하다]",휴게소 마트 여성 전용 주차장 보다 분노 하다
88,무게치는거에 인생버린새끼들,1,"[무게, 치, 인생, 버리다, 새끼]",무게 치 인생 버리다 새끼
...,...,...,...,...
60,그래서 씨발롬아 PC가 나쁘다는 거야 좋다는 거야?,1,"[그래서, 씨발, 롬, PC, 나쁘다, 좋다]",그래서 씨발 롬 PC 나쁘다 좋다
71,아 그런거야? ㅋㅋㅋ,0,"[아, 그렇다, ㅋㅋ, ㅋ]",아 그렇다 ㅋㅋ ㅋ
14,박근혜 안빠는데 보수통합 3원칙 인정함,1,"[박근혜, 안, 빠, 보수, 통합, 원칙, 인정]",박근혜 안 빠 보수 통합 원칙 인정
92,수술도 남자 의사가 잘한다,0,"[수술, 남자, 의사, 잘, 하다]",수술 남자 의사 잘 하다


In [11]:
tokenized_train_sentences = tokenizer(
    list(train_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [12]:
tokenized_test_sentences = tokenizer(
    list(test_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [13]:
print(tokenized_train_sentences[0])        # 0번째 문장에 해당하는 객체 
print(tokenized_train_sentences[0].tokens) # 0번째 문장에 토큰의 목록
print(tokenized_train_sentences[0].ids)    # 0번째 문장에 대한 고유 ID

Encoding(num_tokens=59, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['ĠìłĦ', 'Ġìłľì£¼ëıĦ', 'ĠìĹ¬íĸī', 'Ġê°Ģ', 'ëĭ¤', 'ĠíıŃ', 'íı¬', 'Ġìķŀ', 'ĠìĤ¬ì§Ħ', 'Ġìŀĺ', 'ëĤĺëĭ¤', 'Ġíı¬íĨł', 'Ġì¡´', 'ĠìĤ¬ì§Ħ', 'Ġì°į', 'ëĭ¤', 'Ġê¸°ëĭ¤', 'ë¦¬ëĭ¤', 'Ġìķŀ', 'Ġì§±', 'Ġê²ĮìĿ´', 'Ġì§ľì¦Ŀ', 'ĠëĤ¨', 'ëĭ¤', 'Ġë¨¼ìłĢ', 'Ġê°ģìŀĲ', 'Ġíķľìŀ¥', 'Ġì°į', 'ëĭ¤', 'ĠìĿĮ', 'Ġëĭ¤ìĿĮ', 'Ġë²Ī', 'ê°Ī', 'ëĭ¤', 'Ġì§Ŀ', 'Ġì§ĵ', 'ëĭ¤', 'Ġì°į', 'ëĭ¤', 'Ġìłķë§Ĳ', 'ĠëģĿëĤĺ', 'ëĭ¤', 'Ġì°į', 'ëĭ¤', 'Ġê·¸', 'Ġëĭ¤', 'Ġëª¨', 'ìĿ´ëĭ¤', 'Ġíı¬', 'ì¦Ī', 'Ġë°Ķê¾¸', 'ëĭ¤', 'ĠëĺĲ', 'ĠìĤ¬ì§Ħ', 'Ġì°į', 'ëĭ¤', 'ĠìĿĮ', 'ĠìĿ´ì§Ģ', 'Ġíķĺëĭ¤']
[502, 7050, 3994, 389, 237, 1283, 1520, 1124, 2519, 500, 9178, 26272, 1790, 2519, 1379, 237, 2846, 6527, 1124, 1762, 11350, 3706, 631, 237, 1858, 8840, 20856, 1379, 237, 1677, 1836, 2488, 1340, 237, 4745, 1381, 237, 1379, 237, 939, 5155, 237, 1379, 237, 434, 374, 463, 489, 1526, 2446, 3956, 237, 754, 2519, 1379, 237, 1677, 4798, 3088]


In [ ]:
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encoding, labels):
        self.encodings = encoding
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
        item["labels"] = torch.tensor(self.labels[idx])                             # key: toch.tensor(val[ix])라는 item 딕셔너리 형성
        return item                                                                 # 새로운 labels 키 값에 value torch.tensor(self.labels[idx]) 쌍을 생성
    
    def __len__(self):
        return len(self.labels)                                                     # self.labels 길이 반환

In [14]:
# train_set, test_set에 대한 데이터셋을 각각 생성

train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)   # 사전 학습된 모델 찾아오기
model.to(device)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.ou

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [17]:
# 3 - 2 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = './results',           # 학습결과 저장경로
    num_train_epochs=10,                # 학습 epoch 설정
    per_device_train_batch_size = 8,    # train batch_size 설정
    per_device_eval_batch_size = 64,    # test batch_size 설정
    logging_dir = './logs',             # 학습 log 저장경로
    logging_steps=500,                  # 학습 log 기록 단위
    save_total_limit = 2,               # 학습결과 저장 최대갯수
)

In [18]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 학습과정에서 사용할 평가지표를 위한 함수 설정
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # 정밀도, 재현율, f1 구하기 
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  # 정확도 구하기
  acc = accuracy_score(labels, preds)
  return{
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [19]:
# Trainer 모듈을 사용해 모델의 학습을 컨트롤하은 trainer를 생성
trainer = Trainer(
    model = model,                       # 학습하고자하는 Transformers model
    args=training_args,                  # 위에서 정의한 Trainging Arguments
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [20]:
trainer.train()

c:\Users\GJAISCHOOL\.conda\envs\nv38\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\GJAISCHOOL\AppData\Local\Temp\ipykernel_12548\1167134061.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
100%|██████████| 100/100 [03:11<00:00,  1.91s/it]

{'loss': 0.3424, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 191.4621, 'train_samples_per_second': 3.969, 'train_steps_per_second': 0.522, 'train_loss': 0.3423833465576172, 'epoch': 10.0}


TrainOutput(global_step=100, training_loss=0.3423833465576172, metrics={'train_runtime': 191.4621, 'train_samples_per_second': 3.969, 'train_steps_per_second': 0.522, 'train_loss': 0.3423833465576172, 'epoch': 10.0})

In [21]:
# 평가를 수행하고 custom_ 접두사가 붙은 평가 지표를 출력
results = trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix='custom_')

print(results)

C:\Users\GJAISCHOOL\AppData\Local\Temp\ipykernel_12548\1167134061.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
100%|██████████| 1/1 [00:00<00:00, 82.72it/s]

{'custom__loss': 0.45376983284950256, 'custom__accuracy': 0.8, 'custom__f1': 0.7999999999999999, 'custom__precision': 0.7272727272727273, 'custom__recall': 0.8888888888888888, 'custom__runtime': 0.4949, 'custom__samples_per_second': 40.415, 'custom__steps_per_second': 2.021, 'epoch': 10.0}


In [22]:
# 0: curse, 1: non_curse
def sentence_predict(sent):
    # 평가모드로 변경
    model.eval()

    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 모델이 위치한 GPU로 이동
    # tokenized_sent.to(deivce)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
        )

    # 결과 return
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    sentence = True
    input_sentence = tokenized_sent["input_ids"]
    if result == 0:
        sentence = True

    elif result == 1:
        sentence = False
    return sentence, input_sentence


sentence_predict("씨발이게 맞아?")

(False, tensor([[2295,  640, 3898, 5170,   33]]))

In [23]:
def found_word(input_sentence, found_bad_word):
    result = input_sentence
    if found_bad_word:
        result = badword_find(result)
    return result

def badword_find(input_sentence):
    result = input_sentence
    badword_df = pd.read_excel(r'C:\Users\user\Desktop\X-filter\word_list.xlsx')
    
    found_bad_word = False  # 입력 문장에 단어가 발견되었는지를 나타내는 플래그
    for idx, row in badword_df.iterrows():
        if row["WORD"] in input_sentence:
            # 'WORD'가 입력 문장에 포함된 경우
            new_word = row["대체어"]
            if not pd.isnull(new_word):
                result = result.replace(row["WORD"], new_word)
                found_bad_word = True
            else:
                result = result.replace(row["WORD"], "*" * len(row["WORD"]))
                found_bad_word = True
    
    if not found_bad_word:
        # result = "@" * len(input_sentence)
        result = "혐오 표현입니다."
    return result

# 테스트
input_sentence = "엄마가 죽었으면 좋겠어"
speak_result = badword_find(input_sentence)
print(speak_result)

입


In [24]:
badword_find(input_sentence)

'입'

In [25]:
def speak(sent):
    sentence = sentence_predict(sent)
    if sentence[0] == False:
        return sent
    elif sentence[0] == True:
        return badword_find(sent)

In [26]:
import kss
from pykospacing import Spacing
from soynlp.normalizer import repeat_normalize

# 띄어쓰기 설정
spacing = Spacing()

In [27]:
# 특수문자 제거
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ|a-zA-Z0-9]','', text)
    return text

In [28]:
def clean_and_repeat_normalize(text):
    cleansed_text = cleanse(text)                                             # 특수문자 제거
    normalized_text = repeat_normalize(cleansed_text, num_repeats=2)          # 중복문자 제거
    input_data = re.sub(r'\d', '', normalized_text)                           #  제거
    normalized_text = spacing(input_data)                                     # 띄어쓰기 보정 

    return normalized_text

clean_and_repeat_normalize("아버지가방에들어가신다")

'어바지가 방에 들어가신다'

In [29]:
def final_output():
    input_text = clean_and_repeat_normalize(input())
    sentences = kss.split_sentences(input_text)

    sentences_list = []
    for sentence in sentences:
        text_output = speak(sentence)
        sentences_list.append(text_output)

    long_test = ' '.join(sentences_list)
    print(long_test)
    return long_test

In [42]:
final_output()

지미랄
난리


'난리'